# First (single) level analysis:

In [ ]:
from nilearn.glm.first_level import first_level_from_bids

task_label = "languagelocalizer"
(
    models,
    models_run_imgs,
    models_events,
    models_confounds,
) = first_level_from_bids(
    data.data_dir,                     # path to dataset
    task_label,
    img_filters=[("desc", "preproc")],
    n_jobs=2,
    space_label="",
    sub_labels=["01", "02", "03", "04"],  # comment to run all subjects
)

In [ ]:
from nilearn import plotting

# Assuming you have already run first_level_from_bids
for i, model in enumerate(models):
    print(enumerate(models))
    # Fit the model before accessing design_matrices_
    model.fit(models_run_imgs[i], models_events[i], models_confounds[i])

    print(f"Subject {i+1}: Design Matrix")
    design_matrix = model.design_matrices_[0]  # Design matrix of first run
    plotting.plot_design_matrix(design_matrix)
    # Display the first few rows
    print(design_matrix.head())

In [ ]:
from pathlib import Path

print([Path(run).name for run in models_run_imgs[0]])

In [ ]:
print(models_confounds[0][0].columns)

In [ ]:
print(models_events[0][0]["trial_type"].value_counts())

In [ ]:
from scipy.stats import norm

p001_unc = norm.isf(0.001)

In [ ]:
from math import ceil

import matplotlib.pyplot as plt
import numpy as np
from nilearn import plotting # import plotting here

ncols = 3
nrows = ceil(len(models) / ncols)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8, 4.5))
axes = np.atleast_2d(axes)
model_and_args = zip(models, models_run_imgs, models_events, models_confounds)
for midx, (model, imgs, events, confounds) in enumerate(model_and_args):
    # fit the GLM
    model.fit(imgs, events, confounds)
    # compute the contrast of interest
    zmap = model.compute_contrast("language-string")
    plotting.plot_glass_brain(
        zmap,
        colorbar=False,
        threshold=p001_unc,
        title=f"sub-{model.subject_label}",
        axes=axes[int(midx / ncols), int(midx % ncols)],
        plot_abs=False,
        display_mode="x",
        cmap="bwr",
    )
fig.suptitle("subjects z_map language network (unc p<0.001)")
plotting.show()